In [116]:
import os
import json

import logging

In [117]:
from getpass import getuser

####  Get user  ####
elie = 'elie-medioni'
# path_folder_elie="D://Elie//PhD//Simulation"
path_folder_elie = "D://Elie//PhD//Simulation"
path_LBT_elie = "C://Users//elie-medioni//ladybug_tools//resources//standards//honeybee_standards"

abraham = ''
path_folder_abraham = None
path_LBT_abraham = None

user = getuser()

if user == elie:
    path_LBT_user_defined = path_LBT_elie
    path_folder = path_folder_elie
elif user == abraham:
    path_LBT_user_defined = path_LBT_abraham
    path_folder = path_folder_abraham
else:
    print("The user is not defined")

#### Extract Input file ####
## Path of the input file
path_input_file = os.path.join(path_folder, "Input_Data", "inputs.json")
## load input file
input_file = open(path_input_file, "r")
json_dict = json.load(input_file)
input_file.close()
## Input file variable
epw = json_dict["epw"]
gis = json_dict["gis"]
unit_gis = json_dict["unit_gis"]
target_buildings = json_dict["target_buildings"]
VF_criterion_shading = json_dict["VF_criterion_shading"]
# VF_criterion_simulated_buildings = json_dict["VF_criterion_simulated_buildings"]
# VF_criterion_lwr = json_dict["VF_criterion_lwr"]

#### Prepare paths ####
path_folder_typology = os.path.join(path_folder, "Input_Data", "Typology", "Typologies")
path_folder_construction_and_load_library = os.path.join(path_folder, "Input_Data", "Constructions_and_Loads")
## GIS
path_file_gis = os.path.join(path_folder, "Input_Data", "GIS", gis)
## EPW
path_file_epw = os.path.join(path_folder, "Input_Data", "EPW", epw)
## Simulation parameters
path_folder_simulation_parameter = os.path.join(path_folder, "Input_Data", "Simulation_parameters", "annual_results")
path_simulation_parameter = os.path.join(path_folder_simulation_parameter, "simulation_parameter.json")
## folder outputs
path_folder_output=os.path.join(path_folder, "Program_output", )
## EnergyPlus
path_energyplus_exe = "C:\EnergyPlusV22-1-0\energyplus.exe"

## name of the simulation
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%Hh_%Mm_%Ss")  # get the date and time to get an unique identifier

simulation_name = ""
manual_input = input("type an additional string for the name of the simulation: ")
if manual_input != "":
    simulation_id = "Simulation_" + simulation_name + manual_input
    if simulation_id in os.listdir(path_folder_output):
        simulation_id = "Simulation_" + simulation_name+"_" + manual_input+"_" + dt_string
else:
    simulation_id = "Simulation_" + simulation_name+"_" + manual_input+"_" + dt_string

## Simulation
path_folder_simulation = os.path.join(path_folder_output, simulation_id)
path_folder_context_hbjson = os.path.join(path_folder_simulation, "Context")
path_folder_building_simulation = os.path.join(path_folder_simulation, "Buildings")

In [118]:
print(path_folder_building_simulation)

D://Elie//PhD//Simulation\Program_output\Simulation_BER_proj_R1-W1-G4_s_005\Buildings


In [119]:
from library import Library

## load user defined object for the LBT, before importing the LBT (otherwise the objects will not be updated
Library.load_HBjson_sets(path_construction_and_load_library_folder=path_folder_construction_and_load_library,
                                    path_LBT_user_defined=path_LBT_user_defined)

In [120]:
from urban_canopy import Urban_canopy

# import honeybee
# import honeybee_energy
#
# from honeybee.model import Model
# from honeybee.room import Room
#
# from honeybee_energy import run

In [121]:
## Create Urban Canopy
U_c = Urban_canopy("random_neighborhood")

## Load Typologies in the Urban canopy object
U_c.load_typologies(path_folder_typology)

Typology "BER_proj_R0-W0-G1" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G2" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G3" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G4" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G5" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G6" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W0-G7" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W1-G0" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R0-W2-G0" does not have building type, it will be set as "residential" by default
Typology "BER_proj_R1-W0-G0" does not have building type, it will be set as "residential" b

In [122]:
## Extract data from GIS files
U_c.extract_gis_2D(path_file_gis, unit_gis)

print("GIS extracted")

GIS extracted


In [123]:
## Define buildings to simulate
U_c.select_target_building(target_buildings)

In [124]:
## Create Ladybug geometries with GIS footprint
U_c.create_building_LB_geometry_footprint()
## Create HB room envelop with GIS footprint
U_c.create_building_HB_room_envelop()

In [125]:
# filter context and identify the buildings to simulate

U_c.filter_context(0.1)

In [126]:
print(len(U_c.building_dict[0].context_buildings_id))

0


In [127]:
U_c.building_dict[0].typology = U_c.typology_dict["BER_proj_R1-W1-G4_s_005"]

In [128]:
# print(U_c.building_dict[0].num_floor)

In [129]:
# convert to DF stories and buildings
U_c.create_DF_building_according_to_typology()
# convert to HB model
U_c.convert_DF_building_to_HB_models()
U_c.HB_solve_adjacencies()

In [130]:
## Force rotation on building
# need to guess it and rotate it at the beginning

U_c.building_dict[0].HB_model_force_rotation(180)

In [131]:
# create windows
U_c.assign_conditioned_zone()
U_c.HB_building_window_generation_floor_area_ratio()

# add shades
U_c.add_context_surfaces_to_HB_model()
# assign constructions, loads etc...
U_c.apply_buildings_characteristics()

# add blinds

# add thermal mass
U_c.add_thermal_mass_int_wall()

print("Building modeled")

Building modeled


In [132]:
print(U_c.building_dict[0].HB_model.rooms[0].properties.energy.program_type)
print(U_c.building_dict[0].HB_model.rooms[0].properties.energy.internal_masses[0].construction)


Program Type: IS_Residential_5282_ref_AB
Construction,
 IS_InternalWall_Internalmass_Res_Ref, !- name
 IS_CB41-4-10x50x20-030_half, !- layer 1
 IS_CementMortar_0.015;    !- layer 2


In [133]:
U_c.create_folder_simulation(path_folder_simulation, path_folder_context_hbjson, path_folder_building_simulation)

In [134]:
# All the buildings that are target buildings
U_c.context_to_hbjson(path_folder_context_hbjson)
U_c.context_surfaces_to_hbjson(path_folder_building_simulation)
U_c.GIS_context_individual_to_hbjson(path_folder_building_simulation)

In [135]:
## Merge simulation parameters files
U_c.load_simulation_parameter(path_folder_simulation_parameter, path_simulation_parameter)
## Add design days
U_c.add_design_days_to_simulation_parameters(path_simulation_parameter, path_file_epw)



In [136]:

#### Extract simulation parameters ####
U_c.generate_HB_model(path_folder_building_simulation)




In [137]:
#### Extract simulation parameters ####
U_c.simulate_idf(path_folder_building_simulation, path_simulation_parameter, path_file_epw, path_energyplus_exe)







In [138]:
print("zob")




























zob
